# Monte Carlo Integration

eatimate the integral of a function by averaging random samples of the function's value

> $\int_{L} f(x) \mathrm{d} x \approx F_{N}=\frac{1}{N} \sum_{i=1}^{N} \frac{f\left(x_{i}\right)}{p\left(x_{i}\right)}$
>
> where $X \sim p(x)$

# Path Tracing

```c
// direction wo, from object p
shade(p, wo)
{
    // Russian Roulette
    ramdomly select ksi in a uniform dist in [0, 1];
    // P_RR is a manually specified probability threshold
    if (ksi > P_RR) 
        return .0;
    
    // if choose N directions, rays will explode
    ramdomly choose one directions wi;
    Lo = .0;
    for each wi
    {
        // origin p, direction wi
        trace a ray r(p, wi);
        
        // direct illumination
        if ray r hit the light
            // divide P_RR for (expetion of Lo) = Lo
            // pdf for probability distribution function
            Lo += (1 / N) * L_i * f_r * cosine / pdf(wi) / P_RR;
        
        // indirect illumination
        else if ray r hit an object at q
            Lo += (1 / N) * shade(q, -wi) * f_r * cosine / pdf(wi) / P_RR;
    }
    return Lo;
}            
```

![](./imgs/41.jpg)

```c
ray_generation(cam_pos, pixel)
{
    uniformly choose N sample positions within the pixel;
    pixel_radiance = .0;
    for each sample in the pixel
    {
        shoot a ray r(cam_pos, cam_to_sample);
        if ray r hit the scene at p
            pixel_radiance += 1 / N * shade(p, sample_to_cam);
    }
    return pixel_radiance;
}
```

### more efficient solution: sample integrated point on light

![](./imgs/42.jpg)

> relationship between $\mathrm{d} \omega$ and $\mathrm{d} A$: $d \omega=\frac{d A \cos \theta^{\prime}}{\left\|x^{\prime}-x\right\|^{2}}$
>
> so $\begin{aligned} L_{o}\left(x, \omega_{o}\right) &=\int_{\Omega^{+}} L_{i}\left(x, \omega_{i}\right) f_{r}\left(x, \omega_{i}, \omega_{o}\right) \cos \theta \mathrm{d} \omega_{i_{ }} \\ &=\int_{A} L_{i}\left(x, \omega_{i}\right) f_{r}\left(x, \omega_{i}, \omega_{o}\right) \frac{\cos \theta \cos \theta^{\prime}}{\Vert x^{\prime}-x\Vert ^{2}} \mathrm{~d} A \end{aligned}$
>
> using Monte Carlo Integration, $X \sim pdf=\frac{1}{A}$

```c
shade(p, wo)
{
    // contribution from light source
    uniformly sample the light at x_prime (pdf_light = 1 / A);
    shoot a ray from p to x_prime;
    if the ray is not blocked
        L_dir = L_i * f_r * cosine * cosine_prime / [x_prime - p]^2 / pdf_light;
    
    // contribution from other reflectors
    L_indir = .0;
    test Russian Roulette with probability P_RR;
    uniformly sample the hemisphere toward wi (pdf_hemi = 1 / 2pi);
    trace a ray r(p, wi);
    if ray r hit a non-emitting object at q
        L_indir = shade(q, -wi) * f_r * cosine / pdf_hemi / P_RR;
    return L_dir + L_indir;
}
```